# Welcome to the Multi-Radar/Multi-Sensor System (MRMS) notebook!

#### **Intent:** This notebook will find and visualize MRMS data for a selected location, product, date, and time range.    
#### **Audience:** Anyone with at least 4GB of memory on their computer or computing environment. No programming experience required to go through this notebook, but it will help you understand where this data is coming from.     
#### **Outcome:** A plot showing MRMS imagery from the selected region and product.          

The MRMS system was developed to produce severe weather, transportation, and precipitation products for improved decision-making capability to improve hazardous weather forecasts and warnings, along with hydrology, aviation, and numerical weather prediction.

MRMS is a system with fully-automated algorithms that quickly and intelligently integrate data streams from multiple radars, surface and upper air observations, lightning detection systems, satellite observations, and forecast models. Numerous two-dimensional multiple-sensor products offer assistance for hail, wind, tornado, quantitative precipitation estimations, convection, icing, and turbulence diagnosis. Find more information here: https://www.nssl.noaa.gov/projects/mrms/

# Import statements

In [ ]:
import ipywidgets as widgets
import requests
import s3fs
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import datetime
import numpy.ma as ma
from metpy.plots import ctables
import urllib.request
import gzip
import tempfile

# Definitions

In [ ]:
def aws_region():

    global location_choice
    
    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')   
    
    # Display drop-down menu
    location_choice = widgets.Dropdown(options=[("ALASKA"),("CARIB"),("CONUS"),("GUAM"),("HAWAII")], description='Region:', style=style, layout=layout)
    display(location_choice)

In [ ]:
def aws_product(location):
    
    global product_choice, tilt_choice

    aws = s3fs.S3FileSystem(anon=True)
    data_files = aws.ls(f'noaa-mrms-pds/{location}/', refresh=True)
    
    products = []
    for j in data_files:
        products.append(j[(15 + len(location)):])
        
    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')  
    
    # Display drop-down menu
    product_choice = widgets.Dropdown(options=[("MergedReflectivityQC"),("MergedRhoHV"),("MergedZdr")], description='Radar Product:', style=style, layout=layout)
    tilt_choice =  widgets.Dropdown(options=[("00.50"),("00.75"),("01.00"),("01.25"),("01.50"),("01.75"),("02.00"),("02.25"),("02.50"),("02.75"),("03.00"),("03.50"),("04.00"),("04.50"),("05.00"),("05.50"),("06.00"),("06.50"),("07.00"),("07.50"),("08.00"),("08.50"),("09.00"),("10.00"),("11.00"),("12.00"),("13.00"),("14.00"),("15.00"),("16.00"),("17.00"),("18.00"),("19.00"),], description='Tilt:', style=style, layout=layout)
    display(product_choice)
    display(tilt_choice)

In [ ]:
def aws_date(location, product, tilt):
    
    global year, day, month, shour, smin, ehour, emin
            
    aws = s3fs.S3FileSystem(anon=True)

    # Date options to be shown in the drop-down menus 
    
    year_options = []
    full_product = product + "_" + tilt
    years = aws.ls(f'noaa-mrms-pds/{location}/{full_product}/', refresh=True)
    for y in years:
        year_options.append(y[-8:-4])
            
    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')
    
    # Display drop-down menus
    year = widgets.Dropdown(options=[y for y in np.unique(year_options)], description='Year:', style=style, layout=layout)
    month = widgets.Dropdown(options=[('Jan', '01'), ('Feb', '02'), ('Mar', '03'), ('Apr', '04'), ('May', '05'), ('Jun', '06'), ('Jul', '07'), ('Aug', '08'), ('Sep', '09'), ('Oct', '10'), ('Nov', '11'), ('Dec', '12')], description='Month:', style=style, layout=layout)
    day = widgets.Dropdown(options=[('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31')], description='Day:', style=style, layout=layout)
    shour = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23')], description='Start Hour:', style=style, layout=layout)
    smin = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31'), ('32'), ('33'), ('34'), ('35'), ('36'), ('37'), ('38'), ('39'), ('40'), ('41'), ('42'), ('43'), ('44'), ('45'), ('46'), ('47'), ('48'), ('49'), ('50'), ('51'), ('52'), ('53'), ('54'), ('55'), ('56'), ('57'), ('58'), ('59')], description='Start Minutes:', style=style, layout=layout)
    ehour = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23')], description='End Hour:', style=style, layout=layout)
    emin = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31'), ('32'), ('33'), ('34'), ('35'), ('36'), ('37'), ('38'), ('39'), ('40'), ('41'), ('42'), ('43'), ('44'), ('45'), ('46'), ('47'), ('48'), ('49'), ('50'), ('51'), ('52'), ('53'), ('54'), ('55'), ('56'), ('57'), ('58'), ('59')], description='End Minutes:', style=style, layout=layout)

    start_time = widgets.HBox([shour, smin])
    end_time = widgets.HBox([ehour, emin])

    display(year, month, day)
    display(start_time, end_time)

In [ ]:
def aws_data(location, product, tilt, year, month, day, shour, smin, ehour, emin):
    
    global fnames
    
    datestring = year + month + day
    aws = s3fs.S3FileSystem(anon=True)
    
    full_product = product + "_" + tilt
    data_files = aws.ls(f'noaa-mrms-pds/{location}/{full_product}/{datestring}/', refresh=True)

    fnames = []

    for d in data_files:
        date_time = d[-15:-11]
        if (int(date_time) >= int(shour + smin)) and (int(date_time) <= int(ehour + emin)):
            fnames.append(d[14:])

    print(f"Your search found {len(fnames)} data file(s) that matched your criteria.")

In [ ]:
def aws_plot(location, product):

    for fname in fnames:
        URL = f'https://noaa-mrms-pds.s3.amazonaws.com/{fname}'

        response = urllib.request.urlopen(URL)
        compressed_file = response.read()

        with tempfile.NamedTemporaryFile(suffix=".grib2") as f:
            f.write(gzip.decompress(compressed_file))
            dataset = xr.load_dataset(f.name, engine='cfgrib')

            # Identify plotting coordinates
            lats = dataset.latitude.values
            lons = dataset.longitude.values
            vals = dataset.unknown.values
            vals_nonmiss = ma.masked_where(vals <= 0, vals)

            dataset.close()
            f.close()

        # Set Bounds
        if location == "ALASKA":
            minLat = 54    
            maxLat = 71
            minLon = -173 + 360.
            maxLon = -130 + 360.
            
        elif location == "CARIB":
            minLat = 15    
            maxLat = 21
            minLon = -70 + 360.
            maxLon = -63 + 360.
            
        elif location == "CONUS":
            minLat = 21    
            maxLat = 48
            minLon = -130 + 360.
            maxLon = -60 + 360.
            
        elif location == "GUAM":
            minLat = 11.5    
            maxLat = 15
            minLon = 146.5 
            maxLon = 143 
            
        elif location == "HAWAII":
            minLat = 18    
            maxLat = 28
            minLon = -178 + 360.
            maxLon = -154 + 360.

        title = "MRMS Merged Reflectivity"

        fig = plt.figure(num=1, figsize=(8,5), facecolor='w', edgecolor='k')
        ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.Mercator())
        ax.set_extent([minLon, maxLon, minLat, maxLat], crs=ccrs.Geodetic())
        
        # Set colors
        if product == "MergedReflectivityQC":
            ref_norm, ref_cmap = ctables.registry.get_with_steps('NWSReflectivity', 5, 5)
            units = "Reflectivity (dBZ)"
        if product == "MergedRhoHV":
            ref_norm = None
            ref_cmap = "plasma"
            units = "RhoHV"
        if product == "MergedZdr":
            ref_norm = None
            ref_cmap = "viridis"
            units = "ZDR (dBZ)"
            
        # Add Boundaries
        ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)

        # Plot Data
        radarplot = ax.pcolormesh(lons, lats, vals_nonmiss,transform=ccrs.PlateCarree(),cmap=ref_cmap,norm=ref_norm)
        cbar = plt.colorbar(radarplot)
        cbar.set_label(units)

        date_time = fname[-24:-9]
        scan_time = datetime.datetime.strptime(date_time, "%Y%m%d-%H%M%S")
        plt.title(f"{title}", loc='left', fontweight='bold')
        plt.title('{}'.format(scan_time.strftime('%d %B %Y at %H:%M UTC')), loc='right')

        plt.show()
        plt.close('all')

# Data input here!

In [ ]:
aws_region()

In [ ]:
aws_product(location_choice.value)

In [ ]:
aws_date(location_choice.value, product_choice.value, tilt_choice.value)

In [ ]:
aws_data(location_choice.value, product_choice.value, tilt_choice.value, year.value, month.value, day.value, shour.value, smin.value, ehour.value, emin.value)

In [ ]:
aws_plot(location_choice.value, product_choice.value)

**MRMS Documentation:**
- https://www.nssl.noaa.gov/projects/mrms/ (Product page)
- https://www.nssl.noaa.gov/news/factsheets/MRMS_2015.March.16.pdf (Fact sheet)
- https://mrms.nssl.noaa.gov/qvs/product_viewer/ (Operational product viewer)

**Coding References:**
- https://gitlab.cicsnc.org/jared/mrms-python/-/blob/088e0fac528efdfd2ff19ca912e6ed99bf3783a7/plot_mrms-AVL.py

**Cloud Access:**
- AWS: https://registry.opendata.aws/noaa-mrms-pds/ 

The unique component of this Jupyter notebook is that you are not requried to download any datasets -- all data will be pulled directly from the AWS cloud. You can learn more about NOAA's efforts to move more data to the cloud at this site: https://www.noaa.gov/nodd/datasets. As we continute to make more data widely accessible on the cloud, we'll also create more Jupyter notebooks like this one, so anyone can visualize weather and climate data without any cost or restriction. 